In [7]:
%reload_ext autoreload
%autoreload 2
import os
import yaml
if "NeuroVoice" != os.path.basename(os.getcwd()):
    os.chdir("../")
from pipelines.train import TrainPipeline
from IPython.display import Audio, display
from models.tts.xtts_finetuner import XTTSFinetuner, OUTPUT_SAMPLE_RATE

config_path = "configs/train/shtefanov.yaml"
if not os.path.exists(config_path):
    print("Файл не существует! Исполни команду 'python utils/create_config.py' для создание нового конфига.")
config = yaml.safe_load(open(config_path))
trainpipe = TrainPipeline(config)
trainpipe._preprocess_data()
# trainpipe.train()

По указанному пути '' не найдено .wav файла для обучения. Загрузить файл с Youtube? (да/нет)
(Шаг [1/1]): Укажите ссылку на Youtube-видео:
[youtube] Extracting URL: https://www.youtube.com/watch?v=3NToQdYN3dI
[youtube] 3NToQdYN3dI: Downloading webpage
[youtube] 3NToQdYN3dI: Downloading tv client config
[youtube] 3NToQdYN3dI: Downloading tv player API JSON
[youtube] 3NToQdYN3dI: Downloading web safari player API JSON


[youtube] 3NToQdYN3dI: Downloading m3u8 information
[info] 3NToQdYN3dI: Downloading 1 format(s): 96
[download] Sleeping 5.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1404
[download] Destination: audio.mp4
[download] 100% of    1.42GiB in 00:05:22 at 4.50MiB/s                       
[FixupM3u8] Fixing MPEG-TS in MP4 container of "audio.mp4"
[ExtractAudio] Destination: audio.wav
Deleting original file audio.mp4 (pass -k to keep)
Как назвать .wav файл для обучения? (Например, shtefanov_pope)
Файл 'data/shtefanov/xtts/audio/shtefanov_golodomor.wav' готов для начала обучения!
😞 Mandatory files are not ready.
😞 wavs/ folder is empty.
⚙️ Starting process of creating files... (REQUIRES GPU!)


 25%|█████████▎                            | 722M/2.88G [01:13<03:39, 10.6MiB/s]

In [2]:
trainpipe.train()

😊 All mandatory files are ready!
😊 wavs/ folder is not empty!
😊 All mandatory files are ready!
😊 wavs/ folder is not empty!


/home/egatiyatullin/repos/NeuroVoice/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 594.51it/s]


Starting from base checpoint models/tts/base/coqui/XTTS-v2/model.pth ...


/home/egatiyatullin/repos/NeuroVoice/.venv/lib/python3.10/site-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=m

>> DVAE weights restored from: models/tts/base/coqui/XTTS-v2/dvae.pth
 | > Found 726 files in /home/egatiyatullin/repos/NeuroVoice/data/shtefanov/xtts/shtefanov_pope
0


...


   --> TIME: 2025-10-11 17:26:17 -- STEP: 118/363 -- GLOBAL_STEP: 5200
     | > loss_text_ce: 0.024201367050409317  (0.02808532600243718)
     | > loss_mel_ce: 2.6029350757598877  (2.711566747245142)
     | > loss: 2.627136468887329  (2.7396520719689836)
     | > grad_norm: 0  (0.0)
     | > current_lr: 2.0000000000000003e-06 
     | > step_time: 0.157  (0.15676992222414185)
     | > loader_time: 0.0088  (0.011807191169868081)


   --> TIME: 2025-10-11 17:26:27 -- STEP: 168/363 -- GLOBAL_STEP: 5250
     | > loss_text_ce: 0.030681470409035683  (0.028149661519342943)
     | > loss_mel_ce: 2.4110610485076904  (2.713007128664424)
     | > loss: 2.441742420196533  (2.7411567866802224)
     | > grad_norm: 0  (0.0)
     | > current_lr: 2.0000000000000003e-06 
     | > step_time: 0.1446  (0.1549563606580099)
     | > loader_time: 0.0089  (0.011456533556892754)



[!] Warning: The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.



   --> TIME: 2025-10-11 17:26:38 -- STEP: 218/363 -- GLOBAL_STEP: 5300
     | > loss_text_ce: 0.03041415475308895  (0.028185733312040294)
     | > loss_mel_ce: 3.0713658332824707  (2.7109113740264816)
     | > loss: 3.1017799377441406  (2.739097103613233)
     | > grad_norm: 0  (0.0)
     | > current_lr: 2.0000000000000003e-06 
     | > step_time: 0.1686  (0.15693588431822053)
     | > loader_time: 0.0098  (0.011463265900218157)



[!] Warning: The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.



   --> TIME: 2025-10-11 17:26:48 -- STEP: 268/363 -- GLOBAL_STEP: 5350
     | > loss_text_ce: 0.027538930997252464  (0.02822534652399038)
     | > loss_mel_ce: 2.471592903137207  (2.6950199848680345)
     | > loss: 2.499131917953491  (2.7232453275972346)
     | > grad_norm: 0  (0.0)
     | > current_lr: 2.0000000000000003e-06 
     | > step_time: 0.147  (0.1575836023287987)
     | > loader_time: 0.0101  (0.011502407380004426)


   --> TIME: 2025-10-11 17:26:58 -- STEP: 318/363 -- GLOBAL_STEP: 5400
     | > loss_text_ce: 0.02971404232084751  (0.028169111541981966)
     | > loss_mel_ce: 3.0252764225006104  (2.686058662222617)
     | > loss: 3.054990530014038  (2.714227770484469)
     | > grad_norm: 0  (0.0)
     | > current_lr: 2.0000000000000003e-06 
     | > step_time: 0.1527  (0.15709007011269618)
     | > loader_time: 0.0095  (0.011447212980978146)


 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.030642252415418625 (+0.0003799460828304256)
     | > avg_loss_text

In [6]:
import torchaudio
from IPython.display import Audio, display
wav, sr = torchaudio.load("data/divertito/audio/voice1.wav")
display(Audio(wav, rate=sr))

# 3. Проверяем обученную модель

In [5]:
checkpoint_path="out/shtefanov/xtts/finetuned_model/xtts finetune-October-11-2025_05+05PM-29c9558/best_model.pth"
base_model_path = trainpipe.finetuner.load_base_model()
model = trainpipe.finetuner.load_model(
    trainpipe.device,
    os.path.join(base_model_path,"config.json"),
    checkpoint_path,
    os.path.join(base_model_path, "vocab.json"),
   os.path.join(base_model_path, "speakers_xtts.pth")
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]/home/egatiyatullin/repos/NeuroVoice/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 402.95it/s]


In [10]:
# Generate audio
audio = trainpipe.finetuner.generate_audio(
    model,
    "Это, конечно, потешно блядь. Ты сладкий бубочка, а не уважающий себя мужчина, хахах",
    config["data"]["reference_audio"]
)
display(Audio(audio,  rate = OUTPUT_SAMPLE_RATE))

NameError: name 'OUTPUT_SAMPLE_RATE' is not defined